# Terrain Classification

**One-Time Setup:** *Run the code snippet below once. This sets up your file directory and unzips files. After running for the first time, comment out the code so it isn't run again*

In [ ]:
import zipfile as zf
files = zf.ZipFile("images.zip", 'r')
files.extractall('training/images/')
files.close()
files = zf.ZipFile("ground_truth.zip", 'r')
files.extractall('training/gt/')
files.close()
files = zf.ZipFile("test_images.zip", 'r')
files.extractall('testing/images/')
files.close()
files = zf.ZipFile("test_gt.zip", 'r')
files.extractall('testing/gt/')
files.close()
!cat x* > weights_A.hdf5

!mkdir training/weights
!mkdir prediction
!mkdir prediction/usertrained
!mkdir prediction/pretrained
!mkdir prediction/usertrained/threshold
!mkdir prediction/pretrained/threshold
!mv weights_A.hdf5 training/weights

## Crater Segmentation from Aerial Images 

We're going to walk through the process of implementing, training, and testing out a convolutional neural network for crater segmentation. We will train the neural net on existing and labeled training images. In this case, we'll be working with a dataset of lunar images.

### Unet Introduction

The U-net is a CNN designed specificly for image segmentation. It was developed at the Universtiy of Freiburg, and published in 2015. 

As mentioned in the lecture, the general idea of it is to use a contracting path to compress the image and find features, and then use information from different layers in the network to precisely localize important information. 
You can see how the U-net code is implemented in the Unet.py file. 

If you are curious about more information on how the U-net works, we recommend reading the original paper: https://arxiv.org/pdf/1505.04597.pdf

### Data

Machine learning algorithms require to data from which to learn. The quantity and quality of the dataset can have significant effects on the performance. Let's take a look at some of the images we will be learning from.

![training_image_1](img_1.png)

The darker regions in the photo are created by shadowing where there are craters...

![training_image_0](img_0.png)

In some images, craters may not be present.

![training_image_2](img_3.png)

Or the craters may be challenging to pick out.

### Training

Note that in order to train the neural net, we need to have labeled "truth" to teach the neural net what's correct. An example is shown below. This "truth" image corresponds to the lunar image shown above. We will now train the unet on a set of labeled data. 

![truth_image_2](gt_3.png)

Start by installing some necessary packages, building out the file directory, and importing those packages.

In [ ]:
#install packages
!pip install tensorflow
!pip install keras
!pip install helpers

In [ ]:
#import packages
import numpy as np
import os,sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from unet import *
from augmentation import *
from test_methods import *

### We start by initializing a convolutional neural network based on the U-net architecture. 

The summary shows how the layers are structured with input-shape, functions and number of neurons. You can see that the input has the same shape as the output.

In [ ]:
#create the cnn
model = get_unet()
model.summary()

#### We now declare our variables for training the network


We'll start by training with 10 steps per epoch, and 2 epochs. This is quite short, but as you'll see, we are quite limited computationally.

In [ ]:
#variables for how random images will be generated when training
generator_variables = dict(rotation_range=90,
                           width_shift_range=0.0,
                           height_shift_range=0.0,
                           zoom_range=0.7,
                           horizontal_flip=True,
                           vertical_flip = True,
                           fill_mode='reflect')

#number of images per gradient-calculation
batch_size = 2
#number of times we try gradients per epoch
steps_per_epoch = 10
epochs= 2
#epochs = 1

#### Specify where the training images are stored, and where the model should be saved

In [ ]:
#fetch images
cloud_dir = os.getcwd() 
cloud_dir = cloud_dir + "/"
train_dir = cloud_dir+"training/"
#directory where checkpoint weights will be stored
weights_dir = train_dir+"weights/"
weights = weights_dir+'weights_B.hdf5'
img_folder = 'images'
gtruth_folder = 'gt'

#### Train the model using the generator-variables we previously specified
By using checkpoint the model will be saved in the specified folder

This may take some time. Probably on the order of about 5 minutes per epoch. Go grab a snack and come back. (Or modify the training parameters above to do just one epoch.

In [ ]:
#we make a generator that will be used for training
train_generator = trainGenerator(batch_size,train_dir,img_folder,gtruth_folder,generator_variables)
#we save the weights to a file
checkpoint = ModelCheckpoint(weights, monitor='loss', verbose=1, save_best_only=True)
#we train the model using the generator
model.fit_generator(train_generator,steps_per_epoch=steps_per_epoch,epochs=epochs, callbacks=[checkpoint])

#### We now have a trained model that can be used to perform image segmentation of craters



You'll notice that it takes a lot of time to train the neural net on a regular CPU. As a result, we'll give you a pre-trained model that has been trained for 600 steps per epoch, and 10 epochs. 

(We pre-trained the model for you using Google Colab's GPU. Feel free to try it out yourself!)

### Testing our Model

Let's see how our model performs by giving it data that it has never seen before. This is a different set of images than the training set, since using new images is critical for evaluating whether our model extrapolates well to new data.

#### We declare the name of the file containing the pretrained weights, and where to save the predicted images

In [ ]:
weights = os.getcwd() + '/training/weights/' + 'weights_A.hdf5' # <- this is the pre-trained model
model.load_weights(weights)
save_imgs = True
where_to_save = "prediction/pretrained/"

#### Use generator to make predictions for all images
By using the generator we avoid loading all images to the memory at the same time

In [ ]:
#The generator is created with the name of the folder for the test-images as its parameter
testGene = testGenerator("testing/images")
#run the prediction with the generator, and declare how many images to include.
results = model.predict_generator(testGene,10,verbose=1)
if(save_imgs):
    save_result(where_to_save,results)

### Classifier Output

Run the code below to see the output of one of the prediction images. The neural net outputs a probability for each pixel, based on how confident it is that each pixel is part of a crater edge. In some applications, we may want to have a concrete classification of what is predicted to be a crater edge, and what is not. (In other applications, we may prefer a likelihood.)

In [ ]:
fname = 'prediction/pretrained/1_predict.png'
image = Image.open(fname)#.convert("L")
arr = np.asarray(image)
plt.imshow(arr, cmap='gray')
plt.show()

We will now consider what should count as a crater. Let's define a threshold--the value that each pixel has to exceed in order to be considered a crater edge. (We'll normalize it to a value between 0 and 1.) And let's look at how changing that threshold will affect the results.

The *threshold_prediction* function takes in an image (as a PixelAccess object) and a specified threshold value (from 0 to 1) and outputs the image with each pixel classified as a crater boundary in white, and everything else in black. Remember that the threshold value is normalized to between 0 and 1, but the PixelAccess object contains pixel values between 0 (black) and 65535 (white). Access a pixel with coordinates (i,j) in an image *pix* by pix[i,j].

In [ ]:
def threshold_prediction(pix,thres):
    
    image_size = im.size # Get width and height of image

    WHITE = 65535 #pixel value for white
    BLACK = 0     #pixel value for black
    
    ### BEGIN SOLUTION

    value_thres = thres * WHITE #threshold value as a percentage of the white value required to classify as part of crater edge

    for i in range(image_size[0]):
        for j in range(image_size[1]):
            if pix[i,j] > value_thres:
                pix[i,j] = WHITE
            else:
                pix[i,j] = BLACK

    return pix

    ### END SOLUTION


In [ ]:
"""Check that accuracy functions work correctly"""
from nose.tools import assert_equal
from utils import test_ok

thresholds = [.25,.1]
expected_pix_25 = [[0, 65535, 65535, 0],
 [0, 0, 65535, 65535],
 [0, 0, 65535, 65535],
 [0, 0, 0, 65535],
 [0, 0, 0, 0]]
expected_pix_10 = [[0, 65535, 65535, 65535],
 [0, 65535, 65535, 65535],
 [0, 0, 65535, 65535],
 [0, 0, 0, 65535],
 [0, 0, 0, 0]]
expected_pixs = [expected_pix_25, expected_pix_10]
for i in range(2):
    im = Image.open('prediction/pretrained/8_predict.png')
    im = im.crop((30, 34, 35, 38))
    pix = im.load()
    pix = threshold_prediction(pix,thresholds[i])
    x = expected_pixs[i]
    for i in range(im.size[0]):
        for j in range(im.size[1]):
            assert_equal(x[i][j], pix[i, j])
            x[i][j] = pix[i, j]

test_ok()

In [ ]:
thresholds = [.25,.1]
for i in range(2):
    im = Image.open('prediction/pretrained/1_predict.png') # Can be many different formats.
    #im = Image.open('test_truth/test_gt_1.png') # Can be many different formats.
    pix = im.load() #PixelAccess object
    pix = threshold_prediction(pix,thresholds[i])
    im.save('prediction/pretrained/threshold/1_predict_' + str(i) + '.png')  # Save the modified pixels as .png

Run the code below for the image thresholded at .25

In [ ]:
#![Thresholded_pred](prediction/pretrained/threshold/1_predict_0.png)
fname = 'prediction/pretrained/threshold/1_predict_0.png'
image = Image.open(fname)
arr = np.asarray(image)
plt.imshow(arr, cmap='gray')
plt.show()

And this next image is for a threshold of .1. Notice the presence of more craters, but also thicker edges.

In [ ]:
#![Thresholded_pred](prediction/pretrained/threshold/1_predict_1.png)
fname = 'prediction/pretrained/threshold/1_predict_1.png'
image = Image.open(fname)
arr = np.asarray(image)
plt.imshow(arr, cmap='gray')
plt.show()

Now let's compare it with the "right answer"

In [ ]:
#![Thresholded_pred](testing/gt/test_gt_1.png)
fname = 'testing/gt/test_gt_1.png'
image = Image.open(fname)
arr = np.asarray(image)
plt.imshow(arr, cmap='gray')
plt.show()

Let's view them together

In [ ]:
f, axarr = plt.subplots(1,3)
f.set_figwidth(15)

fname = 'prediction/pretrained/threshold/1_predict_0.png'
image = Image.open(fname)
arr = np.asarray(image)
axarr[0].set_title('threshold .25')
axarr[0].imshow(arr, cmap='gray')

fname = 'prediction/pretrained/threshold/1_predict_1.png'
image = Image.open(fname)
arr = np.asarray(image)
axarr[1].set_title('threshold .1')
axarr[1].imshow(arr, cmap='gray')

fname = 'testing/gt/test_gt_1.png'
image = Image.open(fname)
arr = np.asarray(image)
axarr[2].set_title('Truth')
axarr[2].imshow(arr, cmap='gray')


**Conceptual Question:** Why might 50% confidence not be the right choice? For a Mars mission, would you want to choose a lower or higher threshold? Why? (Hint: Consider the consequences to Mark Watney based on these classifications)

    ### BEGIN SOLUTION
    A: In a planetary exploration mission, risk avoidance is far more important than finding the absolute shortest path.  Therefore, even a 1% chance of an area containing a potentially rover-damaging crater might be too high for us to be willing to drive through that area. We are more inclined to choose a lower threshold for safety critical functions.

    ### END SOLUTION

#### Evaluation metrics
We now want to evaluate the accuracy of the classification at different thresholds. To do that, we need a metric by which to evaluate. Let us consider three different metrics. 

*Overall accuracy* looks at every pixel in an image and asks whether each is classified correctly, as either part of a *crater edge* or *not crater edges*. 

The *true positive rate* is only interested in whether the true craters were detected. In other words, it looks only at the pixels which correspond to *crater edges* in the actual truth data (white pixels), and determines whether the prediction was able to successfully classify those pixels as crater edges.

The *true negative rate* is only interested in areas that are *not crater edges*. This looks only at the black pixels in the truth image, and asks whether each of those non-crater pixels were classified correctly by the prediction image.

Write three functions to evaluate the percentage of pixels classified correctly based on those three metrics. All three functions will take input arguments of:
- a PixelAccess object (see above code for example usage) representing the prediction image, where crater edges are represented by white pixels (value = 65535)
- a second PixelAccess object representing the actual truth data, 
- the size of that PixelAccess object (width and height in pixels). 

All three functions will return a percentage (from 0 to 1):
- The first function, *percentcorrect_all* , will calculate the percentage of correct pixels over the entire image--number of black and white pixels classified correctly. 
- The second function, *percentcorrect_craters* , will calculate the percentage of crater pixels (based on the truth data) that were classified correctly--only looking at the white pixels from the ground truth image and whether the prediction classified those crater pixels correctly. This is the true positive rate. **In images where there are no craters in the truth data, and therefore no white pixels, the % correct of white pixels should return the float NaN**
- The third function, *percentcorrect_noncraters* , will be very similar to the second function, except that you want to calculate the percentage of noncrater pixels that were classified correctly. In other words, this is the true negative rate.

Make sure that the most recent predictions were run with the pre-trained model that we provided to you, as the autograder is expecting the pre-trained model to have been the most recent one used. 

In [ ]:
def percentcorrect_all(pix_pred, pix_truth, image_size):
    
    WHITE = 65535 #pixel value for white
    BLACK = 0     #pixel value for black
    
    ### BEGIN SOLUTION
    
    num_correct_pixels = 0
    for i in range(image_size[0]):
        for j in range(image_size[1]):
            if pix_pred[i,j] == pix_truth[i,j]:
                num_correct_pixels += 1
    return num_correct_pixels/(image_size[0]*image_size[1])

    ### END SOLUTION
    

    
def percentcorrect_craters(pix_pred, pix_truth, image_size):
    
    WHITE = 65535 #pixel value for white
    BLACK = 0     #pixel value for black
    
    ### BEGIN SOLUTION
    
    num_correct_pixels = 0
    num_white_pixels = 0
    for i in range(image_size[0]):
        for j in range(image_size[1]):
            if pix_truth[i,j] == WHITE:
                if pix_pred[i,j] == pix_truth[i,j]:
                    num_correct_pixels += 1
                num_white_pixels += 1
    if num_white_pixels != 0:
        return num_correct_pixels/num_white_pixels
    else:
        return float('NaN')
    
    ### END SOLUTION

def percentcorrect_noncraters(pix_pred, pix_truth, image_size):
    
    WHITE = 65535 #pixel value for white
    BLACK = 0     #pixel value for black
    
    ### BEGIN SOLUTION
    
    num_correct_pixels = 0
    num_black_pixels = 0
    for i in range(image_size[0]):
        for j in range(image_size[1]):
            if pix_truth[i,j] == BLACK:
                if pix_pred[i,j] == pix_truth[i,j]:
                    num_correct_pixels += 1
                num_black_pixels += 1
                
    if num_black_pixels != 0:
        return num_correct_pixels/num_black_pixels
    else:
        return float('NaN')
    
    ### END SOLUTION





Test your code to make sure it's working properly.

In [ ]:
"""Check that accuracy functions work correctly"""
from nose.tools import assert_almost_equal, assert_true
import math
from utils import test_ok

#Test normal image
im = Image.open('prediction/pretrained/' + str(1) +'_predict.png') # Can be many different formats.
im_gt = Image.open('testing/gt/test_gt_' + str(1) + '.png') # Can be many different formats.
pix = im.load()
pix_gt = im_gt.load()
image_size = im.size # Get width and height of image

pix = threshold_prediction(pix,.25)

acc_all = percentcorrect_all(pix, pix_gt, image_size)
acc_craters = percentcorrect_craters(pix, pix_gt, image_size)
acc_noncraters = percentcorrect_noncraters(pix, pix_gt, image_size)

assert_almost_equal(acc_all, 0.871551513671875, delta=0.001)
assert_almost_equal(acc_craters, 0.4389568764568765, delta=0.001)
assert_almost_equal(acc_noncraters, 0.9221604854104173, delta=0.001)

#Test image with no craters
im = Image.open('prediction/pretrained/' + str(0) +'_predict.png') # Can be many different formats.
im_gt = Image.open('testing/gt/test_gt_' + str(0) + '.png') # Can be many different formats.
pix = im.load()
pix_gt = im_gt.load()
image_size = im.size # Get width and height of image

pix = threshold_prediction(pix,.25)

acc_all = percentcorrect_all(pix, pix_gt, image_size)
acc_craters = percentcorrect_craters(pix, pix_gt, image_size)
acc_noncraters = percentcorrect_noncraters(pix, pix_gt, image_size)

assert_almost_equal(acc_all, 1.0, delta=0.001)
assert_true(math.isnan(acc_craters))
assert_almost_equal(acc_noncraters, 1.0, delta=0.001)

test_ok()

#### Examine different thresholds
We will now examine a range of different thresholds and how the accuracy varies for each metric, with 5 of the images we predicted. Feel free to try more (you may need to predict more images first)

In [ ]:
thresholds = [.01, .02, .03, .05, .075, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1] #thresholds to try
n_images = 5 # number of images (already predicted) to evaluate
acc_all = np.zeros([len(thresholds),n_images])
acc_craters = np.zeros([len(thresholds),n_images])
acc_noncraters = np.zeros([len(thresholds),n_images])
mean_acc_all = np.zeros(len(thresholds))
mean_acc_craters = np.zeros(len(thresholds))
mean_acc_noncraters = np.zeros(len(thresholds))

for i in range(len(thresholds)):
    for j in range(n_images):
        im = Image.open('prediction/pretrained/' + str(j) +'_predict.png') # Can be many different formats.
        im_gt = Image.open('testing/gt/test_gt_' + str(j) + '.png') # Can be many different formats.
        pix = im.load()
        pix_gt = im_gt.load()
        image_size = im.size # Get width and height of image

        pix = threshold_prediction(pix,thresholds[i])
        acc_all[i][j] = percentcorrect_all(pix, pix_gt, image_size)
        acc_craters[i][j] = percentcorrect_craters(pix, pix_gt, image_size)
        acc_noncraters[i][j] = percentcorrect_noncraters(pix, pix_gt, image_size)
        
    mean_acc_all[i] = np.nanmean(acc_all[i])
    mean_acc_craters[i] = np.nanmean(acc_craters[i])
    mean_acc_noncraters[i] = np.nanmean(acc_noncraters[i])
    

f, axarr = plt.subplots(1,2)
f.set_figwidth(15)

axarr[0].plot(thresholds,mean_acc_all,'r')
axarr[0].plot(thresholds,mean_acc_craters,'c')
axarr[0].plot(thresholds,mean_acc_noncraters,'m')
axarr[0].set_ylabel('Accuracy')
axarr[0].set_xlabel('Threshold')
axarr[0].legend(['Overall','Craters','NonCraters'])
axarr[0].set_title('Detecting Craters: Accuracy of Different Methods')

axarr[1].plot(thresholds,1-mean_acc_noncraters)
axarr[1].plot(thresholds,1-mean_acc_craters)
axarr[1].set_ylabel('Missed Detection Rate')
axarr[1].set_xlabel('Threshold')
axarr[1].legend(['Missed Craters','Missed NonCraters'])
axarr[1].set_title('Missed Detections: Accuracy of Different Methods')


In the left plot, we look at the accuracy of detection using the three metrics we defined above. The red line is the overall accuracy (rate of correct detection for both crater and noncrater pixels). The cyan line shows the rate of detection of craters, and the magenta line shows the rate of detection of noncraters (areas that are safe).

However, we can also look at the same information from a different perspective--that of missed detections. In the right plot, we look at the rate of missed detections for craters, and the rate of misclassification of noncrater (safe) regions. The rate of missed detection is simply (1 - the rate of detection).

#### Receiver Operating Characteristic Curve

Another way to view this is the Receiver Operating Characteristic (ROC) curve, shown below. The ROC curve plots the rate of positive detection against the rate of false positives. This allows a direct comparison of the tradeoffs between the rate of true crater detections and false alarms. We'll also plot a similar one for missed detections by comparing false negatives to false positives. This compares the rate of missed craters with the rate of missed safe regions. Both plots essentially tell the same story, but from different perspectives.

In [ ]:
f, axarr = plt.subplots(1,2)
f.set_figwidth(15)

axarr[0].plot(1-mean_acc_noncraters,mean_acc_craters)
axarr[0].set_xlabel('False Positive Rate')
axarr[0].set_ylabel('True Positive Rate')
axarr[0].set_title('ROC Curve: Crater Detection vs. False Alarms')


axarr[1].plot(1-mean_acc_noncraters,1-mean_acc_craters)
axarr[1].set_xlabel('False Positive Rate')
axarr[1].set_ylabel('False Negative Rate')
axarr[1].set_title('ROC Curve: Missed Detection vs. Missed Safe Regions')


**Conceptual Question:** Explain the trends that you see in the above plots. How would you select the threshold to be used? Why might using the overall accuracy (across black and white pixels) to determine which threshold to use be a good or bad idea?

    ### BEGIN SOLUTION
    As the threshold decreases, the percentage of craters detected increases, but accuracy of non-crater regions decreases. This illustrates the tradeoff between being conservative and classifying non-crater regions accurately. The ROC curve shows that there will always be a tradeoff. A higher rate of crater detection means a lower rate of noncrater detection. One method of selecting the threshold may be to find the "knee" in the graph, where the derivative are equal. Another method would be to set a requirement for crater accuracy, and adjusting the threshold to meet that. [Answers may vary.] However, using the overall accuracy is not a good metric, as most of the pixels are black, so a naive guess that everything is not a crater may appear to have high overall accuracy, but result in catastrophic consequences when a rover crashes into a crater.

    ### END SOLUTION

## Other Terrain Classification Methods

In addition to terrain classification using aerial images, the lecture also discussed using images taken by the rover and using vibration and velocity signals from the rover's physical interaction with the terrain to do terrain classification.  The lecture also touched on combining these lower-level classifiers into a more accurate meta-classifier.  These final few conceptual questions will highlight some of the key takeaways from these other techniques.

#### Ground-Based Image Classification

We discussed a few machine learning techniques that use color, texture, and geometry to differentiate between terrain types like sand and rock.

**Conceptual Question:** Why is color alone not a robust enough visual classifier on Mars?

    ### BEGIN SOLUTION
    A:  Illumination can change r,g,b values so training on one light setting could cause the classifier to fail in different lighting conditions.  Additionally there is very little color variation on Mars.

    ### END SOLUTION

A geometric hazard refers to something like a rock that has a defined geometric shape.  The rover tries to avoid large rocks because, obviously, running into a rock could be hazardous to the rover.

**Conceptual Question:** What is an example of a non-geometric hazard and why might detection of these types of hazards be beneficial to detect?

    ### BEGIN SOLUTION
    An example could be loose sand on a steep slope; we want to detect this because this could lead to significant slipping and the rover could get stuck


    ### END SOLUTION

#### Tactile Classification

For this section, we discussed how to process vibration and velocity signals from the rover's wheels to be able to determine the type of ground the rover is driving over.

**Conceptual Question:** When might tactile-based classification be beneficial over image-based classification?

    ### BEGIN SOLUTION
    A: When situations not represented in the training set are encountered. When lighting is poor or vision is obscured for some reason (i.e. a dust storm), or you need to classify the ground directly beneath the rover instead of in front of the rover for some reason

    ### END SOLUTION

**Conceptual Question:** In what situations might tactile-based classification fail?

    ### BEGIN SOLUTION
    A: When situations not represented in the training set are encountered. For tactile-based classification, this could mean the rover is traversing terrain that was not examined fully in the training set or it's traveling at a speed or with a load for which data was not gathered in the training set.
    ### END SOLUTION

#### Classifier Fusion

The last section of the lecture discussed combining the earlier classifiers to build a stronger classifier.

**Conceptual Question:** Describe at least one benefit of using ground-based image classification and tactile classification in conjunction with one another.

    ### BEGIN SOLUTION
    Different methods may have higher accuracies on different terrain or in different conditions, and using multiple methods may provide for taking advantage of each classifier where they are more accurate. Additionally, the tactile classification methods may provide labels and truth data for training visual classifiers, allowing the rover to learn new terrain from experience without human supervision.

    ### END SOLUTION